In [ ]:
from common import *
import rasterio

use_hvplot()

In [ ]:
client = start_cluster(5, 12)
client

# Snow Depth comparison between Snotel and CBRFC Snow-17 (HRRR F06)

In [ ]:
start_year = 2017
end_year = 2021

time=slice(f"{start_year -1}-10-01", f"{end_year}-07-31")

## CBRFC

In [ ]:
ALEC2HLF = 1
ALEC2HMF = 2
ALEC2HUF = 3

INCH_TO_MM = 25.4

In [ ]:
with rasterio.open(CBRFC_DIR / 'ERW_CBRFC_zones.tif') as zones:
    zone_data = zones.read(1)

In [ ]:
zone_data[zone_data==241] = 0

In [ ]:
CBRFC_CSV = pd.read_csv(
    CBRFC_DIR / 'SNOW17ALEC2_2018to2021.csv',
    parse_dates=True,
    index_col=0,
    header=0,
    names=['ALEC2HLF', 'ALEC2HMF', 'ALEC2HUF', 'Average'],
    dtype={'ALEC2HLF': np.float64, 'ALEC2HMF': np.float64, 'ALEC2HUF': np.float64},
)
CBRFC_CSV.ALEC2HLF *= INCH_TO_MM
CBRFC_CSV.ALEC2HMF *= INCH_TO_MM
CBRFC_CSV.ALEC2HUF *= INCH_TO_MM

## iSnobal 

In [ ]:
wy_swe = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy20*' / 'erw/*/snow.nc').as_posix(),
    parallel=True,
)

In [ ]:
wy_swe.coords['mask'] = (('y', 'x'), zone_data)

## Compare CBRFC to iSnobal

In [ ]:
wy_swe.specific_mass.where(wy_swe.mask == ALEC2HUF).mean(("x", "y")). \
    hvplot(label='ALEC HUF (iSnobal)', color='indigo'). \
    opts(
        title='iSnobal versus Snow-17 CBRFC SWE', ylabel='SWE (mm)',
        yformatter='%d',
        width=1280, height=640
    ) * \
wy_swe.specific_mass.where(wy_swe.mask == ALEC2HMF).mean(("x", "y")).hvplot(label='ALEC HMF (iSnobal)', color='teal') * \
wy_swe.specific_mass.where(wy_swe.mask == ALEC2HLF).mean(("x", "y")).hvplot(label='ALEC HLF (iSnobal)', color='gold') * \
CBRFC_CSV.ALEC2HUF.plot(color='mediumorchid', label='ALEC HUF (CBRFC)', line_dash='dashed') * \
CBRFC_CSV.ALEC2HMF.plot(color='cadetblue', label='ALEC HMF (CBRFC)', line_dash='dashed') * \
CBRFC_CSV.ALEC2HLF.plot(color='orange', label='ALEC HLF (CBRFC)', line_dash='dashed')

In [ ]:
wy_swe.specific_mass.where(wy_swe.mask == ALEC2HUF).mean(("x", "y")). \
    hvplot(label='ALEC HUF', color='indigo'). \
    opts(
        title='iSnobal SWE', ylabel='SWE (mm)',
        yformatter='%d',
        width=1280, height=640
    ) * \
wy_swe.specific_mass.where(wy_swe.mask == ALEC2HMF).mean(("x", "y")).hvplot(label='ALEC HMF', color='teal') * \
wy_swe.specific_mass.where(wy_swe.mask == ALEC2HLF).mean(("x", "y")).hvplot(label='ALEC HLF', color='gold') 

In [ ]:
CBRFC_CSV.ALEC2HUF.plot(color='indigo', line_dash='dashed'). \
    opts(title='Snow-17 CBRFC SWE', ylabel='SWE (mm)', width=1280, height=640) * \
CBRFC_CSV.ALEC2HMF.plot(color='teal', line_dash='dashed') * \
CBRFC_CSV.ALEC2HLF.plot(color='gold', line_dash='dashed')

In [ ]:
wy_swe.specific_mass.where(wy_swe.mask == ALEC2HUF).mean(("time")).plot(size=10)

In [ ]:
wy_swe.specific_mass.where(wy_swe.mask == ALEC2HMF).mean(("time")).plot(size=10)

In [ ]:
wy_swe.specific_mass.where(wy_swe.mask == ALEC2HLF).mean(("time")).plot(size=10)